In [60]:
def duplicated_varnames(df):
    """Return a dict of all variable names that 
    are duplicated in a given dataframe."""
    repeat_dict = {}
    var_list = list(df) # list of varnames as strings
    for varname in var_list:
        # make a list of all instances of that varname
        test_list = [v for v in var_list if v == varname] 
        # if more than one instance, report duplications in repeat_dict
        if len(test_list) > 1: 
            repeat_dict[varname] = len(test_list)
    return repeat_dict

for dataframe in dataframes:
    print(duplicated_varnames(dataframe))

{}
{}
{}
{}
{}
{}
{}
{'Nacionalidad / Etnia': 2}
{'Nacionalidad / Etnia': 2}
{'Nacionalidad / Etnia': 2}


In [65]:
import pandas as pd

files = [
    'data/Copia-de-femicidios_2010-1.xlsx',
    'data/Copia-de-femicidios_2011.xlsx',
    'data/Copia-de-femicidios_2012.xlsx',
    'data/Copia-de-femicidios_2013.xlsx',
    'data/Copia-de-femicidios2014final.xlsx',
    'data/Femicidios-2015-1-2.xlsx',
    'data/Femicidios-2016.xls',
    'data/Femicidios 2017 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2017.csv',
    'data/Femicidios 2018 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2018.csv',
    'data/Femicidios 2019 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2019.csv'
]
line_1_files = [
    'data/Femicidios-2016.xls',
    'data/Femicidios 2017 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2017.csv',
]
cols_rename = {
    'Antecedentes Judiciales': 'Antecedentes',
    'Antecedentes judiciales': 'Antecedentes',
    'Categoria delito ': 'Categoria Delito',
    'Relación ': 'Relación',
    'Categoria Delito/Ley Femic.': 'Categoría delito/ Ley Femicidio',
    'Denuncia/ M cautelar': 'Denuncia/M cautelar',
    #'Ocupación ': 'Ocupación',
    'Edad ': 'Edad',
    #'Ocupación.1': 'Ocupación',
    'Edad .1': 'Edad homicida',
    'Antecedentes ': 'Antecedentes',
    'Fecha y Lugar': 'Fecha y lugar',
    #'Nacionalidad / Etnia.1': 'Nacionalidad / Etnia'
}
cols_delete = [
    'Unnamed: 15',
    'Unnamed: 16',
    'Unnamed: 0'
]
dataframes = []
for file in files:
    if file in line_1_files:
        header_line = 1
    else:
        header_line = 2
    if file.split('.')[-1] == 'csv':
        current_df = pd.read_csv(file, header=header_line)
    else:
        current_df = pd.read_excel(file, header=header_line)
    for column in current_df:
        for col_rename in cols_rename:
            if column == col_rename:
                current_df.rename(columns={column: cols_rename[column]}, inplace=True)
        for col_delete in cols_delete:
            if column == col_delete:
                del current_df[column]
    current_df['source'] = file
    dataframes.append(current_df)
from collections import Counter

counter = Counter()
for dataframe in dataframes:
    counter.update(dataframe.columns)
counter

Counter({'Fecha y lugar': 7,
         'Región': 10,
         'Nombre': 10,
         'Edad': 10,
         'Ocupación': 10,
         'Hecho': 10,
         'Vsexual': 10,
         'Tipo Femicidio': 10,
         'Femicidia': 6,
         'Edad.1': 9,
         'Relación': 10,
         'Suicidio': 10,
         'Ocupación ': 3,
         'Antecedentes': 10,
         'Denuncia/M cautelar': 10,
         'Categoria Delito': 2,
         'Sentencia': 10,
         'source': 10,
         'Sernam': 8,
         'Categoría delito/ Ley Femicidio': 8,
         'Ocupación.1': 7,
         'Situación judicial': 7,
         'Tribunal': 7,
         'Femicida': 4,
         'Edad homicida': 1,
         'Prensa': 1,
         'Fecha': 3,
         'Lugar': 3,
         'Nacionalidad / Etnia': 3,
         'Nacionalidad / Etnia.1': 3,
         'Unnamed: 24': 1,
         'Unnamed: 25': 1,
         'Información medios': 2,
         'Información medios 2': 2})

In [64]:
#for dataframe in dataframes:
#    print(dataframe.shape)
pd.concat(dataframes, sort=False).to_csv('preview.csv', index=False)

In [62]:
pd.read_csv(line_1_files[-1], header=2)

,1,15/01/2017,Puerto Montt,Los Ríos,Sonia Riquelme Alvarado,69,Unnamed: 6,chilena,Unnamed: 8,se presume,...,chileno,9 condenas VIF,Unnamed: 18,Unnamed: 19,parricidio,prisión preventiva,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,2.0,16/01/2017,Maipú,Metropolitana,Camila Yáñez Briceño,28.0,NaN,chilena,Encontrada en la tina de su casa. Pareja llama...,NaN,...,chileno,tenía cautelar - prohibición de acercarse,NaN,NaN,femicidio,prisión preventiva,NaN,NaN,NaN,NaN
1,3.0,20/01/2017,"Llolleo, San Antonio",Valparaíso,Iris de las Mercedes Peralta Moraga,63.0,NaN,chilena,NaN,NaN,...,chileno,NaN,NaN,NaN,femicidio,NaN,NaN,NaN,NaN,NaN
2,4.0,24/01/2017,Melipilla,Metropolitana,Silvia Ninaja Condori,NaN,NaN,boliviana,descuartizada; esposo escapa con hija de 5 año...,NaN,...,boliviano,autor confeso,NaN,NaN,femicidio,prófugo; se entregó en Bolivia; formalizado en...,NaN,NaN,http://www.chvnoticias.cl/policial/hombre-que-...,NaN
3,5.0,29/01/2017,San Ramón,Metropolitana,Liliana Hurtado Echeverry,49.0,NaN,colombiana,3 puñaladas,NaN,...,colombiano,Familiares indican tenía antecedentes en Colombia,NaN,NaN,femicidio,prisión preventiva,NaN,NaN,"Los medios no informan el caso, sólo se sabe p...",Información en Colombia: http://www.noticiasrc...
4,6.0,05/02/2017,Coyhaique,Aysen,Ruth María Erices Aniñir,35.0,NaN,chilena,estrangulada,NaN,...,chileno,3 condenas por VIF,NaN,NaN,femicidio,NaN,NaN,NaN,http://www.emol.com/noticias/Nacional/2017/02/...,https://elchelenko.cl/2018/01/30/rebajan-la-co...
5,7.0,06/02/2017,"Carrizal, San Javier",Maule,Carmen Vásquez Contreras,42.0,NaN,chilena,golpes con hacha,NaN,...,chileno,NaN,NaN,NaN,femicidio,NaN,NaN,NaN,NaN,NaN
6,8.0,08/02/2017,San Bernardo,Metropolitana,Maribel del Carmen Gormaz Ibacache,26.0,NaN,chilena,escopetazo en la cabeza,NaN,...,chileno,Agresor convivía con la víctima hace seis mese...,NaN,NaN,femicidio,detenido,NaN,NaN,NaN,NaN
7,9.0,15/02/2017,La Pintana,Metropolitana,Jaritza Figueroa Vargas,20.0,NaN,chilena,balazo,NaN,...,chileno,NaN,NaN,NaN,femicidio,NaN,NaN,NaN,NaN,NaN
8,10.0,23/02/2017,Pemuco/Ñuble,Bio Bio,Andrea Riquelme Toledo,34.0,NaN,chilena,brutal agresión,NaN,...,chileno,NaN,NaN,NaN,femicidio,NaN,NaN,NaN,NaN,NaN
9,11.0,28/02/2017,La Serena,Coquimbo,Daniela Valentina Reyes Espejo,17.0,NaN,chilena,envenenamiento y golpe en la cabeza. PDI y fam...,NaN,...,chileno,NaN,NaN,NaN,femicidio,se investiga,NaN,NaN,http://www.diarioeldia.cl/policial/muerte-jove...,http://maderofm.com/peritajes-realizados-por-p...
